<a href="https://www.inove.com.ar"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/PA%20Banner.png" width="1000" align="center"></a>


# NLP - Bot basado en reglas con Tensorflow
Este ejemplo consiste en armar BOT simple basado en una red neuronal con Tensorflow

v2.0

In [1]:
import json
import string
import random 
import numpy as np

import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout

import pickle

# Recolectar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline1.png" width="1000" align="middle">

In [2]:
# Dataset en formato JSON que representa las posibles preguntas (patterns)
# y las posibles respuestas por categoría (tag)
data = {"intents": [
             {"tag": "bienvenida",
              "patterns": ["Hola", "¿Cómo estás?", "¿Qué tal?"],
              "responses": ["Hola!", "Hola, ¿Cómo estás?"],
             },
             {"tag": "nombre",
              "patterns": ["¿Cúal es tu nombre?", "¿Quién sos?"],
              "responses": ["Mi nombre es MarvelBOT", "Yo soy MarvelBOT"]
             },
            {"tag": "contacto",
              "patterns": ["contacto", "número de contacto", "número de teléfono", "número de whatsapp", "whatsapp"],
              "responses": ["Podes contactarnos al siguiente número +54-9-11-2154-4777", "Contactonos al whatsapp número +54-9-11-2154-4777"]
             },
            {"tag": "envios",
              "patterns": ["¿Realizan envios?", "¿Cómo me llega el paquete?"],
              "responses": ["Los envios se realizan por correo, lo enviaremos a la dirección que registraste en la página"]
             },
            {"tag": "precios",
              "patterns": ["precio", "Me podrás pasar los precios", "¿Cuánto vale?", "¿Cuánto sale?"],
              "responses": ["En el catálogo podrás encontrar los precios de todos nuestros productos en stock"]
             },
            {"tag": "pagos",
              "patterns": ["medios de pago", "tarjeta de crédito", "tarjetas", "cuotas"],
              "responses": ["Contactanos al whatsapp número +54-9-11-2154-4777 para conocer los beneficios y formas de pago vigentes"]
             },
            {"tag": "stock",
              "patterns": ["Esto está disponible", "¿Tenes stock?", "¿Hay stock?"],
              "responses": ["Los productos publicados están en stock"]
             },
            {"tag": "agradecimientos",
              "patterns": [ "Muchas gracias", "Gracias"],
              "responses": ["Por nada!, cualquier otra consulta podes escribirnos"]
             },
             {"tag": "despedida",
              "patterns": [ "Chau", "Hasta luego!"],
              "responses": ["Hasta luego!", "Hablamos luego!"]
             }
]}

# Procesar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline2.png" width="1000" align="middle">

### Herramientas de preprocesamiento de datos
Entre las tareas de procesamiento de texto en español se implementa:
- Quitar números
- Quitar símbolos de puntuación
- Quitar caracteres acentuados

In [3]:
import re
import string

# El preprocesamento en castellano requiere más trabajo

def preprocess_clean_text(text):
    # pasar a minúsculas
    text = text.lower()
    # quitar números
    pattern = r'[0-9\n]'
    text = re.sub(pattern, '', text)
    # quitar caracteres de puntiación
    text = ''.join([c for c in text if c not in (string.punctuation+"¡"+"¿")])
    # quitar caracteres con acento
    text = re.sub(r'[àáâä]', "a", text)
    text = re.sub(r'[éèêë]', "e", text)
    text = re.sub(r'[íìîï]', "i", text)
    text = re.sub(r'[òóôö]', "o", text)
    text = re.sub(r'[úùûü]', "u", text)
    return text

In [4]:
# En castellano utilizamos estos dos signos además de los genéricos del inglés
string.punctuation + "¡" + "¿"

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~¡¿'

In [5]:
# Demostración de como funciona el procesos de preprocesamiento
preprocess_clean_text("¿cómo5!")

'como'

### Lematizacion
La lematización es el proceso de llevar una palabra a su raiz (a la palabra raiz del diccionario).
- Primero debemos contar una tabla de transformación de lematización, en donde por cada palabra de nuestro vocabulario nos retorne la palabra raiz (lookup table).
- Este proceso nos permitirá reducir la cantidad de palabras distintas que tendrá que manejar el bot (acotar el vocablario).

In [6]:
import os
import gdown
if os.access('lematizacion-es.pickle', os.F_OK) is False:
    if os.access('lematizacion-es.zip', os.F_OK) is False:
        url = 'https://drive.google.com/uc?id=16leuM9PuFXAkmw34XeQy-84h8WGAYxJw&export=download'
        output = 'lematizacion-es.zip'
        gdown.download(url, output, quiet=False)
    !unzip -q lematizacion-es.zip
else:
    print("El archivo ya se encuentra descargado")

Downloading...
From: https://drive.google.com/uc?id=16leuM9PuFXAkmw34XeQy-84h8WGAYxJw&export=download
To: /content/lematizacion-es.zip
100%|██████████| 5.42M/5.42M [00:00<00:00, 101MB/s]


In [7]:
with open("lematizacion-es.pickle",'rb') as fi:
    lemma_lookupTable = pickle.load(fi)

In [8]:
# Ejemplo del funcionamiento de lematización
palabras_ensayo = ["estar", "estoy", "estás", "está", "estamos" ,"estais", "estan", "estaremos", "estuvieron"]
for palabra in palabras_ensayo:
    print(f"{palabra} -> {lemma_lookupTable.get(palabra)}")

estar -> estar
estoy -> estar
estás -> estar
está -> estar
estamos -> estar
estais -> estar
estan -> estar
estaremos -> estar
estuvieron -> estar


### Preprocesado del dataset

In [9]:
words = []
classes = []
doc_X = []
doc_y = []
# Tokenizar cada "pattern" y agregar cada palabra al vocabulario (vocabulary)
# Los tokens que se toman de cada pattern se agrega a doc_X
# Cada tag se agrega a doc_y
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        # trasformar el patron a tokens
        tokens = preprocess_clean_text(pattern).split(" ")
        # lematizar los tokens
        lemma_words = []
        for token in tokens:
            lemma = lemma_lookupTable.get(token)
            if lemma is not None:
                lemma_words.append(lemma)
            else:
                print("UNK:", token)
        
        if not lemma_words:
            continue
        
        words += lemma_words
        doc_X.append(pattern)
        doc_y.append(intent["tag"])
    
    # Agregar el tag a las clases
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

# Elminar duplicados con "set" y ordenar el vocubulario y las clases por orden alfabético
vocab = sorted(set(words))
classes = sorted(set(classes))
len(vocab)

UNK: que
UNK: sos
UNK: me
UNK: me
UNK: hasta
UNK: luego


38

# Explorar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline3.png" width="1000" align="middle">

In [10]:
print("vocab:", vocab)
print("classes:", classes)
print("doc_X:", doc_X)
print("doc_y:", doc_y)

vocab: ['adios', 'comer', 'contacto', 'credito', 'cual', 'cuanto', 'cuota', 'dar', 'disponible', 'el', 'envio', 'este', 'gracia', 'haber', 'hola', 'llegar', 'los', 'medio', 'mucho', 'nombre', 'numero', 'pago', 'paquete', 'pasar', 'precio', 'pudrir', 'quien', 'realizar', 'salir', 'ser', 'stock', 'tal', 'tarjeta', 'telefono', 'tener', 'tu', 'valer', 'whatsapp']
classes: ['agradecimientos', 'bienvenida', 'contacto', 'despedida', 'envios', 'nombre', 'pagos', 'precios', 'stock']
doc_X: ['Hola', '¿Cómo estás?', '¿Qué tal?', '¿Cúal es tu nombre?', '¿Quién sos?', 'contacto', 'número de contacto', 'número de teléfono', 'número de whatsapp', 'whatsapp', '¿Realizan envios?', '¿Cómo me llega el paquete?', 'precio', 'Me podrás pasar los precios', '¿Cuánto vale?', '¿Cuánto sale?', 'medios de pago', 'tarjeta de crédito', 'tarjetas', 'cuotas', 'Esto está disponible', '¿Tenes stock?', '¿Hay stock?', 'Muchas gracias', 'Gracias', 'Chau']
doc_y: ['bienvenida', 'bienvenida', 'bienvenida', 'nombre', 'nombre

In [11]:
doc_y_encoded = [classes.index(label) for label in doc_y]
doc_y_encoded

[1, 1, 1, 5, 5, 2, 2, 2, 2, 2, 4, 4, 7, 7, 7, 7, 6, 6, 6, 6, 8, 8, 8, 0, 0, 3]

# Entrenar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline4.png" width="1000" align="middle">

In [13]:
X_train = []
y_train = []

for doc, label in zip(doc_X, doc_y_encoded):
    # Transformar la pregunta (input) en tokens y lematizar
    lemma_words = []
    tokens = preprocess_clean_text(doc).split(" ")
    for token in tokens:
        lemma = lemma_lookupTable.get(token)
        if lemma is not None:
            lemma_words.append(lemma)

    # Transformar los tokens en "Bag of words" (arrays de 1 y 0)
    bow = []
    for word in vocab:
        bow.append(1) if word in lemma_words else bow.append(0)

    # Crear el array de salida (class output) correspondiente
    output_row = list([0] * len(classes))
    output_row[label] = 1

    
    print("X:", bow, "y:", output_row)
    X_train.append(bow)
    y_train.append(output_row)

X_train = np.array(X_train)
y_train = np.array(y_train)

X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] y: [0, 1, 0, 0, 0, 0, 0, 0, 0]
X: [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] y: [0, 1, 0, 0, 0, 0, 0, 0, 0]
X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0] y: [0, 1, 0, 0, 0, 0, 0, 0, 0]
X: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0] y: [0, 0, 0, 0, 0, 1, 0, 0, 0]
X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] y: [0, 0, 0, 0, 0, 1, 0, 0, 0]
X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] y: [0, 0, 1, 0, 0, 0, 0, 0, 0]
X: [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [14]:
X_train.shape

(26, 38)

In [15]:
y_train.shape

(26, 9)

In [19]:
input_shape = X_train.shape[1]
output_shape = y_train.shape[1]
print("input:", input_shape, "output:", output_shape)

input: 38 output: 9


In [20]:
# Entrenamiento del modelo DNN
# - Modelo secuencial
# - Con regularización
# - softmax y optimizador Adam
model = Sequential()
model.add(Dense(128, activation="relu", input_shape=(input_shape,)))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(output_shape, activation = "softmax"))

model.compile(loss='categorical_crossentropy',
              optimizer="Adam",
              metrics=["accuracy"])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               4992      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 9)                 585       
                                                                 
Total params: 13,833
Trainable params: 13,833
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
hist = model.fit(x=X_train, y=y_train, epochs=200, verbose=1)

Epoch 1/200
1/1 [==============================] - 1s 766ms/step - loss: 2.2429 - accuracy: 0.1154
Epoch 2/200
1/1 [==============================] - 0s 13ms/step - loss: 2.1483 - accuracy: 0.1154
Epoch 3/200
1/1 [==============================] - 0s 8ms/step - loss: 2.1796 - accuracy: 0.1154
Epoch 4/200
1/1 [==============================] - 0s 16ms/step - loss: 2.1550 - accuracy: 0.1538
Epoch 5/200
1/1 [==============================] - 0s 12ms/step - loss: 2.2253 - accuracy: 0.1154
Epoch 6/200
1/1 [==============================] - 0s 9ms/step - loss: 2.1380 - accuracy: 0.1923
Epoch 7/200
1/1 [==============================] - 0s 12ms/step - loss: 2.1560 - accuracy: 0.1154
Epoch 8/200
1/1 [==============================] - 0s 13ms/step - loss: 2.1604 - accuracy: 0.1538
Epoch 9/200
1/1 [==============================] - 0s 16ms/step - loss: 2.1224 - accuracy: 0.2308
Epoch 10/200
1/1 [==============================] - 0s 9ms/step - loss: 2.1113 - accuracy: 0.1154
Epoch 11/200
1/1 [===

# Utilizar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline6.png" width="1000" align="middle">

In [22]:
responses = [[""]] * len(classes)
for intent in data["intents"]:
    responses[classes.index(intent["tag"])] = intent["responses"]

responses

[['Por nada!, cualquier otra consulta podes escribirnos'],
 ['Hola!', 'Hola, ¿Cómo estás?'],
 ['Podes contactarnos al siguiente número +54-9-11-2154-4777',
  'Contactonos al whatsapp número +54-9-11-2154-4777'],
 ['Hasta luego!', 'Hablamos luego!'],
 ['Los envios se realizan por correo, lo enviaremos a la dirección que registraste en la página'],
 ['Mi nombre es MarvelBOT', 'Yo soy MarvelBOT'],
 ['Contactanos al whatsapp número +54-9-11-2154-4777 para conocer los beneficios y formas de pago vigentes'],
 ['En el catálogo podrás encontrar los precios de todos nuestros productos en stock'],
 ['Los productos publicados están en stock']]

In [23]:
for i in range(5):
    message = input("")
    print("Q:", message)
    # preprocesamiento + lematizacion
    # ------------------------------------------
    # Transformar la pregunta (input) en tokens y lematizar
    lemma_words = []
    tokens = preprocess_clean_text(message).split(" ")
    for token in tokens:
        lemma = lemma_lookupTable.get(token)
        if lemma is not None:
            lemma_words.append(lemma)

    # Transformar los tokens en "Bag of words" (arrays de 1 y 0)
    bow = []
    for word in vocab:
        bow.append(1) if word in lemma_words else bow.append(0)
    # ------------------------------------------

    probs = model.predict([bow])
    score = probs.max()
    if score > 0.4:  # threshold 0.4        
        index = probs.argmax(axis=1)[0]
        result = random.choice(responses[index])
        print(f"BOT [{score:.2f}]: {result}")
    else:
        print(f"BOT [{score:.2f}] Perdon, no comprendo la pregunta.")

Hola
Q: Hola
BOT [0.87]: Hola!
Quiero hacer un pedido
Q: Quiero hacer un pedido
BOT [0.16] Perdon, no comprendo la pregunta.
Cuanto vale
Q: Cuanto vale
BOT [0.99]: En el catálogo podrás encontrar los precios de todos nuestros productos en stock
Gracias!
Q: Gracias!
BOT [0.92]: Por nada!, cualquier otra consulta podes escribirnos
TU nombre es?
Q: TU nombre es?
BOT [0.93]: Mi nombre es MarvelBOT


# Descargar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline7.png" width="1000" align="middle">

In [24]:
# Exportar los datos importants (vocabulario, clases, el bot y el dataset utilizado)
import pickle
import json

pickle.dump(vocab, open('vocab.pkl','wb'))
pickle.dump(responses, open('responses.pkl','wb'))
model.save('bot.h5')

In [25]:
# Comprimir todos los datos necesarios
!zip -r bot_data.zip bot.h5 vocab.pkl responses.pkl lematizacion-es.pickle

  adding: bot.h5 (deflated 29%)
  adding: vocab.pkl (deflated 40%)
  adding: responses.pkl (deflated 37%)
  adding: lematizacion-es.pickle (deflated 77%)


In [26]:
from google.colab import files
files.download('bot_data.zip') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>